In [ ]:
import cv2

In [ ]:
#!pip install -q -U tensorflow>=1.8.0
#!pip install foolbox
#!pip install tqdm

import tensorflow as tf
import foolbox

import talos as ta
from keras.models import Sequential
from keras.layers import Dense, Layer, Dropout, Activation
from keras import backend as K
from talos.model.early_stopper import early_stopper
from keras.callbacks import ModelCheckpoint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

In [ ]:
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

print(x_train.shape[0], 'train set')
print(x_test.shape[0], 'test set')

fashion_mnist_labels = ["digit 0",  
                        "digit 1",      
                        "digit 2",     
                        "digit 3",        
                        "digit 4",         
                        "digit 5",       
                        "digit 6",         
                        "digit 7",       
                        "digit 8",          
                        "digit 9"]   

img_index = 1
label_index = y_train[img_index]
print ("y = " + str(y_train[5]) + " " +(fashion_mnist_labels[label_index]))
plt.imshow(x_train[img_index])

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])

print(x_train.shape)
print(x_test.shape)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [ ]:
print("Number of train data : " + str(len(x_train)))
print("Number of test data : " + str(len(x_test)))

In [ ]:
(x_train, x_val) = x_train[10000:], x_train[:10000] 
(y_train, y_val) = y_train[10000:], y_train[:10000]

print(x_train.shape)
print(x_test.shape)


In [ ]:
y_classes = y_test
# One-hot encode the labels
nb_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_val = tf.keras.utils.to_categorical(y_val, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)
print(x_train.shape[0], 'train set')
print(x_val.shape[0], 'validation set')
print(x_test.shape[0], 'test set')
print(y_classes)

In [ ]:
class BioSigmoid(Layer):
    def __init__(self, a, b, r, **kwargs):
        super(BioSigmoid, self).__init__(**kwargs)
        self.a = K.cast_to_floatx(a)
        self.b = K.cast_to_floatx(b)
        self.r = K.cast_to_floatx(r)
        
    def call(self, inputs):
        return self.a + (self.b - self.a)*K.sigmoid(self.r * inputs)


    def get_config(self):
        config = {'a': float(self.a), 'b': float(self.b), 'r': float(self.r)}
        base_config = super(BioSigmoid, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

    
    
class BioSoftmax(Layer):
    def __init__(self, a, b, r, axis=-1, **kwargs):
        super(BioSoftmax, self).__init__(**kwargs)
        self.supports_masking = True
        self.axis = axis
        self.a = K.cast_to_floatx(a)
        self.b = K.cast_to_floatx(b)
        self.r = K.cast_to_floatx(r)

    def call(self, inputs):
        return self.a + (self.b - self.a)*K.softmax(self.r * inputs, axis=self.axis)

    def get_config(self):
        config = {'axis': self.axis}
        base_config = super(BioSoftmax, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape


In [ ]:
def build_model(x_train, y_train, x_val, y_val, params):
    
    print(params)
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    model = Sequential()
    model.add(Dense(x_train.shape[1], input_dim = x_train.shape[1]))
    model.add(BioSigmoid(a=params['a'], b=params['b'], r=params['r'])) 
    model.add(Dense(10))
    model.add(BioSigmoid(a=params['a'], b=params['b'], r=params['r']))
    model.add(Dense(10))
    model.add(BioSoftmax(a=params['a'], b=params['b'], r=params['r']))  
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=False)

    out = model.fit(x_train, y_train,
                    batch_size=params['batch_size'],
                    epochs=params['epochs'],
                    callbacks=[checkpointer,early_stopper(params['epochs'], patience=10)],
                    verbose = 1,
                    validation_data=[x_val, y_val])
    
    return out, model



### Optimize with Talos here ...

In [ ]:
x = np.concatenate((x_train, x_val), axis=0)
y = np.concatenate((y_train, y_val), axis=0)

In [ ]:
p = {'a': np.linspace(0.0,0.15,10),
     'b': np.linspace(0.85,1.0,10),
     'r': np.linspace(0.5,5.1,5),
     'batch_size': [25],
     'epochs': [1]
}

In [ ]:

h = ta.Scan(x, y, params=p,
            model=build_model,
            grid_downsample=.15,
            dataset_name='keras-!mnist_adv',
            experiment_no='9')

In [ ]:
h.data.head()

In [ ]:
l = ta.Reporting(h)
print(l.high())
print(l.best_params())

In [ ]:
df = pd.read_csv('keras-!mnist_adv.csv')

In [ ]:
df = df.round(4)
print(df.sort_values('val_acc', ascending=False))

In [ ]:
def build_best_model():
    
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    best_a = 0.13
    best_b = 0.9333
    best_r = 2.8
    
    model = Sequential()
    model.add(Dense(x_train.shape[1], input_dim = x_train.shape[1]))
    model.add(BioSigmoid(a = best_a, b = best_b, r = best_r))
    model.add(Dense(10))
    model.add(BioSigmoid(a = best_a, b = best_b, r = best_r))
    model.add(Dense(10))
    model.add(BioSoftmax(a = best_a, b = best_b, r = best_r))  
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    
    return model

def build_default_model():
    
    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    best_a = 0.0
    best_b = 1.0
    best_r = 1.0
    
    model = Sequential()
    model.add(Dense(x_train.shape[1], input_dim = x_train.shape[1]))
    model.add(BioSigmoid(a = best_a, b = best_b, r = best_r))
    model.add(Dense(10))
    model.add(BioSigmoid(a = best_a, b = best_b, r = best_r))
    model.add(Dense(10))
    model.add(BioSoftmax(a = best_a, b = best_b, r = best_r))  
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
model_best = build_best_model()
model_def = build_default_model()

In [ ]:
model_best.summary()

### Train the model

In [ ]:
checkpointer = ModelCheckpoint(filepath='model_best.weights.best.hdf5', verbose = 1, save_best_only=False)
model_best.fit(x_train,
         y_train,
         batch_size=25,
         epochs=1,
         validation_data=(x_val, y_val),
         callbacks=[checkpointer])

In [ ]:
checkpointer = ModelCheckpoint(filepath='model_def.weights.best.hdf5', verbose = 1, save_best_only=False)
model_def.fit(x_train,
         y_train,
         batch_size=25,
         epochs=1,
         validation_data=(x_val, y_val),
         callbacks=[checkpointer])

### Test the model

In [ ]:
# Evaluate the model on test set
score_best = model_best.evaluate(x_test, y_test, verbose=0)
score_def = model_def.evaluate(x_test, y_test, verbose=0)

print('\n', 'Test accuracy (best):', score_best[1])
print('\n', 'Test accuracy (default):', score_def[1])

### Load the model and generate adversary

In [ ]:

foolbox_model_best = foolbox.models.TensorFlowModel.from_keras(model_best,bounds = (0.0, 1.0))
foolbox_model_def = foolbox.models.TensorFlowModel.from_keras(model_def,bounds = (0.0, 1.0))

from collections import Counter

w, h = 28, 28

target = 4

### :: best model

In [ ]:
target_norm_best = []
target_adv_best = []
pred_norm_best = []
pred_adv_best = []

attack_fgsm = foolbox.attacks.FGSM(foolbox_model_best)
count = 0
sum_best = 0

for i in range(1,len(y_classes)):
    if(y_classes[i]==target):
        print("Normal")
        image = x_test[i]
        target_norm_best.append(np.argmax(foolbox_model_best.predictions(image)))
        pred = model_best.predict(image.reshape(1,x_test.shape[1]))
        pred_norm_best.append(pred[0][y_classes[i]])
        
        print("Adversarial")
        adversarial = attack_fgsm(image,label = y_classes[i])
        target_adv_best.append(np.argmax(foolbox_model_best.predictions(adversarial)))                        
        adver_pred = model_best.predict(adversarial.reshape(1,x_test.shape[1]))
        pred_adv_best.append(adver_pred[0][y_classes[i]])

        count+=1
        sum_best+=(target_norm_best != target_adv_best)
        print("-.-", count)


### :: default model

In [ ]:
target_norm_def = []
target_adv_def = []
pred_norm_def = []
pred_adv_def = []

attack_fgsm = foolbox.attacks.FGSM(foolbox_model_def)
count = 0
sum_def = 0

for i in range(1,len(y_classes)):
    if(y_classes[i]==target):
        print("Normal")
        image = x_test[i]
        target_norm_def.append(np.argmax(foolbox_model_def.predictions(image)))
        pred = model_def.predict(image.reshape(1,x_test.shape[1]))
        pred_norm_def.append(pred[0][y_classes[i]])
        
        print("Adversarial")
        adversarial = attack_fgsm(image,label = y_classes[i])
        target_adv_def.append(np.argmax(foolbox_model_def.predictions(adversarial)))                        
        adver_pred = model_def.predict(adversarial.reshape(1,x_test.shape[1]))
        pred_adv_def.append(adver_pred[0][y_classes[i]])

        count+=1
        sum_def+=(target_norm_def != target_adv_def)
        print("-.-", count)

### :: print and save results

In [ ]:
print("Bio algorithm")
print("Target:", target)
print("Number of misclassified examples:", sum_best)
print("Number of target examples:", count)
print(Counter(target_norm_best),"norm")
print(Counter(target_adv_best),"adv")

print("-.-")

print("Default algorithm")
print("Target:",target)
print("Number of misclassified examples:", sum_def)
print("Number of target examples:", count)
print(Counter(target_norm_def),"norm")
print(Counter(target_adv_def),"adv")

cols = ['target_norm_best','target_adv_best',
        'target_norm_def','target_adv_def'
        'pred_norm_best','pred_norm_def',
        'misclass_num_best','misclass_num_def', "target_num_class",
        'test_acc_best','test_acc_adv']
df_def = pd.DataFrame({"target_norm_best" : target_norm_best,
                       "target_adv_best" : target_adv_best,
                       "target_norm_def" : target_norm_def, 
                       "target_adv_def" : target_adv_def,
                       "pred_adv_best" : pred_adv_best,
                       "pred_adv_def" : pred_adv_def,
                       "misclass_num_best" : np.ones(len(pred_adv_best))*sum_best,
                       "misclass_num_def" : np.ones(len(pred_adv_def))*sum_def,
                       "target_num_class" : np.ones(len(pred_adv_def))*count,
                       "test_acc_best": np.ones(len(pred_adv_best))*score_best[1],
                       "test_acc_def": np.ones(len(pred_adv_def))*score_def[1]
                      })
df_def.to_csv('keras-!mnist_adv-results'+str(target)+'.csv',index=False)
